This notebook will be used to redesign the tidal ellipse functions to make them more user friendly

In [26]:
import os
import datetime
import matplotlib.pylab as plt
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex
import csv
import pandas as pd
import seaborn as sns
from dateutil import tz

import netCDF4 as nc
from salishsea_tools import (viz_tools, tidetools as tt)
from salishsea_tools.nowcast import (analyze, research_VENUS)

%matplotlib inline

In [301]:
def fittit(uaus, time):
    """Function to find tidal components from a tidal current component
        across a specified area of the grid at a single depth, at a
        single point through the water column or a sigle depth averaged
        grid point. Must perform twice, once for each tidal current vector
        in order to complete the analysis.
        In order to calculate the tidal components of an area at a single
        depth the velocity vector must only have 3 dimensions. For a depth
        profile it must onyl have 2 dimensions and for a single point and depth
        it must be a float.
        ***[time, depth, x, y]

    :arg uaus: One of the orthogonal tidal current velocities.
    :type uaus:  :py:class:'np.ndarray' or float

    :arg time: Time over which the velocitie were being taken in seconds.
    :type time: :py:class:'np.ndarray'

    :arg imin: Minimum i value in the grid that will be evaluated for tidal
        ellipses. (Must be between 1 and 308, default 0)
    :type imin: float

    :arg imax: Maximum i value in the grid that will be evaluated for tidal
         ellipses. (Must be between 1 and 398, default 0)
    :type imax: float

    :arg jmin: Minimum j value in the grid that will be evaluated for tidal
        ellipses. (Must be between 1 and 498, default 0)
    :type jmin: float

    :arg jmax: Maximum j value in the grid that will be evaluated for tidal
         ellipse. (Must be between 1 and 498, default 0)

    :arg dj: Interval along the j direction

    :returns M2amp, M2pha, K1amp, K1pha:
        The amplitude and phase lag of each tidal component (M2 and K1)
        of a single tidal velocity vector.

    """
    #Case 1: a time series of velocities with depth at a single location.
    if uaus.ndim == 2:

        thesize = uaus.shape[1]
        M2amp = np.zeros(thesize)
        M2pha = np.zeros(thesize)
        K1amp = np.zeros(thesize)
        K1pha = np.zeros(thesize)
        
        #Calculates the parameters for one depth and one location at a time from its time series
        for dep in np.arange(0, len(uaus[1])-1):
            if uaus[:, dep].any() != 0:
                fitted, cov = tt.curve_fit(tt.double, time[:], uaus[:, dep])
                fitted[0], fitted[1] = tt.convention_pha_amp(fitted[0], fitted[1])
                fitted[2], fitted[3] = tt.convention_pha_amp(fitted[2], fitted[3])
                M2amp[dep] = fitted[0]
                M2pha[dep] = fitted[1]
                K1amp[dep] = fitted[2]
                K1pha[dep] = fitted[3]

    #Case 2 : a time series of an area of velocities at a single depth
    elif uaus.ndim == 3:
        thesize = (uaus.shape[1], uaus.shape[2])
        M2amp = np.zeros(thesize)
        M2pha = np.zeros(thesize)
        K1amp = np.zeros(thesize)
        K1pha = np.zeros(thesize)

        for i in np.arange(0, uaus.shape[1]):
            for j in np.arange(0, uaus.shape[2]):
                if uaus[:, i, j].any() != 0.:
                    fitted, cov = tt.curve_fit(tt.double, time[:], uaus[:, j, i])
                    fitted[0], fitted[1] = tt.convention_pha_amp(
                        fitted[0], fitted[1])
                    fitted[2], fitted[3] = tt.convention_pha_amp(
                        fitted[2], fitted[3])
                    M2amp[j, i] = fitted[0]
                    M2pha[j, i] = fitted[1]
                    K1amp[j, i] = fitted[2]
                    K1pha[j, i] = fitted[3]
    
    #Case 3: a time series of an area of velocities with depth
    elif uaus.ndim == 4:
        thesize = (uaus.shape[1], uaus.shape[2], uaus.shape[3])
        M2amp = np.zeros(thesize)
        M2pha = np.zeros(thesize)
        K1amp = np.zeros(thesize)
        K1pha = np.zeros(thesize)

        for dep in np.arange(0, u.shape[1]):
            for i in np.arange(0, u.shape[2]):
                for j in np.arange(0, u.shape[3]):
                    if uaus[:,dep, i, j].any() != 0.:
                        fitted, cov = tt.curve_fit(tt.double, time[:], uaus[:, dep, j, i])
                        fitted[0], fitted[1] = tt.convention_pha_amp(
                            fitted[0], fitted[1])
                        fitted[2], fitted[3] = tt.convention_pha_amp(
                            fitted[2], fitted[3])
                        M2amp[dep, j, i] = fitted[0]
                        M2pha[dep, j, i] = fitted[1]
                        K1amp[dep, j, i] = fitted[2]
                        K1pha[dep, j, i] = fitted[3]

    #Case 4: a time series of a single location with a single depth.
    else:
        M2amp = 0
        M2pha = 0
        K1amp = 0
        K1pha = 0

        if uaus[:].any() != 0.:
            fitted, cov = tt.curve_fit(tt.double, time[:], uaus[:])
            fitted[0], fitted[1] = tt.convention_pha_amp(fitted[0], fitted[1])
            fitted[2], fitted[3] = tt.convention_pha_amp(fitted[2], fitted[3])
            M2amp = fitted[0]
            M2pha = fitted[1]
            K1amp = fitted[2]
            K1pha = fitted[3]

    return M2amp, M2pha, K1amp, K1pha

In [337]:
def ellipse_files_nowcast(to, tf, i, j, path, depthrange='None'):
    """ Gets the filenames from the nowcasts and extracts the u and v velocities.
    **only works for single point now
    """
    #Makes a list of the filenames that follow the criteria in the indicated path.
    filesu = analyze.get_filenames(to, tf, '1h', 'grid_U', path) 
    filesv = analyze.get_filenames(to, tf, '1h', 'grid_V', path)
    
    u, timer = analyze.combine_files(
        filesu, 'vozocrtx', depthrange, j, i)

    v, timee = analyze.combine_files(
        filesv, 'vomecrty', depthrange,  j, i)

    #For the nowcast the reftime is always Sep10th 2014. Set time of area we are looking at relative to this time.
    reftime = datetime.datetime(2014, 9, 10, tzinfo=tz.tzutc())
    time = tt.convert_to_seconds(timer, reftime=reftime)
    dep = nc.Dataset(filesu[-1]).variables['depthu']
    return u, v, time, dep

In [340]:
def prepare_vel(u, v, depav='None', dep='None'):
    """Preparing the time series of the orthogonal pair of velocities to get tidal ellipse parameters. 
    This function masks, rotates and unstaggers the time series as well as depth averaging if 
    
    :arg u: One of the orthogonal tidal current velocities. Must be already prepared for the analysis.
    :type u:  :py:class:'np.ndarray'
    
    :arg v: One of the orthogonal tidal current velocities. Must be already prepared for the analysis.
    :type v:  :py:class:'np.ndarray'
    
    :arg time: Time over which the velocities were taken; in seconds.
    :type time: :py:class:'np.ndarray'

    :arg dep: depth vector corresponding to the depth of the velocities. Default is 'None' for a single depth.
    :type dep: :py:class:'np.ndarray' or string
    
    :arg tidecorr: Tidal corrections in aplitude and phase. Default is the nowcast values. 
    :type tidecorr: dictionary
    """
    #Masks land values
    u_0 = np.ma.masked_values(u, 0)
    v_0 = np.ma.masked_values(v, 0)

    #Unstaggers velocities.
    u_u, v_v = research_VENUS.unstag_rot(u_0, v_0)

    #Depth averages the velocties if applicable
    if depav != 'None':
        #Find the depths between the limits set.
        j = np.where(np.logical_and(dep > depav[0], dep < depav[1]))
        u_slice = u_u[:, j[0]]
        v_slice = v_v[:, j[0]]
        dep = dep[j]

        u_u = analyze.depth_average(u_slice, dep, 1)
        v_v = analyze.depth_average(v_slice, dep, 1)
    return u_u, v_v

In [328]:
def get_params(u, v, time, dep='None', tidecorr=research_VENUS.CorrTides):
    """Calculates tidal ellipse parameters from the u and v time series. Maintains their shape the shape 
    of the velocities enters only loosing the time dimensions.
    
    :arg u: One of the orthogonal tidal current velocities. Must be already prepared for the analysis.
    :type u:  :py:class:'np.ndarray'
    
    :arg v: One of the orthogonal tidal current velocities. Must be already prepared for the analysis.
    :type v:  :py:class:'np.ndarray'
    
    :arg time: Time over which the velocities were taken; in seconds.
    :type time: :py:class:'np.ndarray'

    :arg dep: depth vector corresponding to the depth of the velocities. Default is 'None' for a single depth.
    :type dep: :py:class:'np.ndarray' or string
    
    :arg tidecorr: Tidal corrections in aplitude and phase. Default is the nowcast values. 
    :type tidecorr: dictionary
    """
    #Running fittit to get the amplitude and phase of the velcity time series.
    uM2amp, uM2pha, uK1amp, uK1pha = fittit(u, time)
    vM2amp, vM2pha, vK1amp, vK1pha = fittit(v, time)

    #Tide corrections from a dictionary
    uM2pha = uM2pha + tidecorr['M2']['uvt']
    uK1pha = uK1pha + tidecorr['K1']['uvt']
    vM2pha = vM2pha + tidecorr['M2']['uvt']
    vK1pha = vK1pha + tidecorr['K1']['uvt']

    uM2amp = uM2amp * tidecorr['M2']['ft']
    uK1amp = uK1amp * tidecorr['K1']['ft']
    vM2amp = vM2amp * tidecorr['M2']['ft']
    vK1amp = vK1amp * tidecorr['K1']['ft']
    
    #Converting from u/v amplitude and phase to ellipe parameters.
    CX, SX, CY, SY, ap, am, ep, em, maj, mi, the, pha = tt.ellipse_params(
        uM2amp, uM2pha, vM2amp, vM2pha)

    CXk, SXk, CYk, SYk, apk, amk, epk, emk, majk, mik, thek, phak = tt.ellipse_params(
        uK1amp, uK1pha, vK1amp, vK1pha)
    
    #Saving all the useful parameters in a matrix. The shape of the parameters is based on the
    # u and v variables but averaged over time.
    #For one location, one depth
    if u.ndim == 1:
        params = np.zeros((8))
        params[:] = [maj, mi, the, pha, majk, mik, thek, phak]
        
    #For a depth profile of a location
    elif u.ndim == 2:
        params = np.zeros((9, len(maj)))
        for i in np.arange(0,len(maj)-1):
            params[:,i] = [dep[i], maj[i], mi[i], the[i], pha[i], majk[i], mik[i], thek[i], phak[i]]
    
    #For an area at a single depth.
    else:
        params = np.zeros((8, maj.shape[0], maj.shape[1]))
        for i in np.arange(0,maj.shape[0]):
            for j in np.arange(0,maj.shape[1]):
                params[:,i,j] = [maj[i,j], mi[i,j], the[i,j], pha[i,j], majk[i,j], mik[i,j], thek[i,j], phak[i,j]]
    return params

In [251]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2015,7,10)
tf=datetime.datetime(2015,7,11)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

In [338]:
def tidalellipse(to, tf, i, j, path, depav='None'):
    u, v, time, dep = ellipse_files_nowcast(to, tf, i, j, path)
    uall, vall = prepare_vel(u,v, i, j)
    return uall,vall

In [352]:
i = np.arange(266,270)
j = np.arange(312,318)


u, v, time, dep = ellipse_files_nowcast(to, tf, i, j, path)
print u.shape

(48, 40, 6, 4)


In [356]:
rang = np.arange(0,4)
rang2 = np.arange(0,2)
print u[:,:,rang, rang2].shape

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (4,) (2,) 

In [341]:
unow = np.empty((48,40, 2,2))
vnow = np.empty((48,40, 2,2))

for x, ind in zip(i, [0,1]):
    for y, dex in zip(j, [0,1]):
        u, v = tidalellipse(to,tf,x,y, path)
        unow[:,:,ind,dex] = u
        vnow[:,:, ind,dex] = v

UnboundLocalError: local variable 'j' referenced before assignment

In [ ]:
viz_tools.unstagger(